# FIFA VALUE OF PLAYER PREDICTION 

18 May 2022

                                                                                    Team Members 
                                                                                     * Siddhant Gupta (50441839)
                                                                                     * Kishore Kumar (50442203)
                                                                                     * Vineeth ReddyTati (50369160)
                                                                                     * Anjali T (50441491)   

Our project considered one year data of fifa players , fifa 2018  . We intended to predict the value of the player based on features provided  . To establish this we used two models . Namely 
- linear regression 
- k fold method 
- SVC 
- Random Forest 
- Descision Tree 
We then plotted the predicted values correspoinding to the actual value to understand the accuracy of our models . 


We followed the below listed steps to create our model
- loading the data using pandas 
- understanding the data and columns 
- normalizing the data into understandable data bases 
- visualising the data 
- understanding the dependencies for value
- pre processing data , removing na's , clearing less useful features 
- dividing the table into training and testing data set to understand the accuracy of our system 
- constructing the models 
- visualising the models 

In [ ]:
### loading all necesary libraries 
import pandas as pd
import sqlite3
from sqlite3 import Error
import seaborn as sns
import numpy as np
import matplotlib as plt 
%matplotlib inline
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##  1) Creating data base and tables from text file 

In [ ]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows



In [ ]:
sofifa_id, age, overall, potential, value_eur, wage_eur, international_reputation = [], [], [], [], [], [], []

header = None
with open('players_18.txt', encoding = 'utf-8') as file:
    for line in file:
        if not line.strip():
            continue 
        if not header:
            header = line.strip().split('\t')
            continue    
        line = line.strip().split('\t')
        sofifa_id.append(line[0])
        age.append(line[4])
        overall.append(line[10])
        potential.append(line[11])
        value_eur.append(line[12])
        wage_eur.append(line[13])
        international_reputation.append(line[16])
        
df = []
for i in range(len(sofifa_id)):
    df.append((sofifa_id[i], age[i], overall[i], potential[i], value_eur[i], wage_eur[i], international_reputation[i]))  

In [ ]:
database_filename = 'project.db'
    
conn = sqlite3.connect(database_filename)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS df")
cur.execute("CREATE TABLE IF NOT EXISTS df (sofifa_id INTEGER NOT NULL, age INTEGER NOT NULL, overall INTEGER NOT NULL, potential INTEGER NOT NULL, value_eur INTEGER NOT NULL, wage_eur INTEGER NOT NULL, international_reputation INTEGER NOT NULL);")
cur.executemany("INSERT INTO df (sofifa_id, age, overall, potential, value_eur, wage_eur, international_reputation) VALUES (?,?,?,?,?,?,?);", df)
conn.commit()

In [ ]:
select = 'select * from df'
df = pd.read_sql_query(select, conn)
display(df)

In [ ]:
sofifa_id, age, overall, potential, value_eur, wage_eur, player_positions, international_reputation, weak_foot, skill_moves, release_clause_eur, contract_valid_until = [], [], [], [], [], [], [], [], [], [], [], []

header = None
with open('players_18.txt', encoding = 'utf-8') as file :
    for line in file:
        if not line.strip():
            continue
        if not header:
            header = line.strip().split('\t')
            continue   
        line = line.strip().split('\t')
        sofifa_id.append(line[0])
        age.append(line[4])
        overall.append(line[10])
        potential.append(line[11])
        value_eur.append(line[12])
        wage_eur.append(line[13])
        player_positions.append(line[14].strip('"'))
        international_reputation.append(line[16])
        weak_foot.append(line[17])
        skill_moves.append(line[18])
        release_clause_eur.append(line[22])
        contract_valid_until.append(line[28])
        
dt = []
for i in range(len(sofifa_id)):
    dt.append((sofifa_id[i], age[i], overall[i], potential[i], value_eur[i], wage_eur[i], player_positions[i], international_reputation[i], weak_foot[i], skill_moves[i], release_clause_eur[i], contract_valid_until[i]))

In [ ]:
database_filename = 'project.db'
    
conn = sqlite3.connect(database_filename)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS dt")
cur.execute("CREATE TABLE IF NOT EXISTS dt (sofifa_id INTEGER NOT NULL, age INTEGER NOT NULL, overall INTEGER NOT NULL, potential INTEGER NOT NULL, value_eur INTEGER NOT NULL, wage_eur INTEGER NOT NULL, player_positions TEXT NOT NULL, international_reputation INTEGER NOT NULL, weak_foot INTEGER NOT NULL, skill_moves INTEGER NOT NULL, release_clause_eur INTEGER NOT NULL, contract_valid_until INTEGER NOT NULL);")
cur.executemany("INSERT INTO dt (sofifa_id, age, overall, potential, value_eur, wage_eur, player_positions, international_reputation, weak_foot, skill_moves, release_clause_eur, contract_valid_until) VALUES (?,?,?,?,?,?,?,?,?,?,?,?);", dt)
conn.commit()

In [ ]:
select = 'select * from dt'
dt = pd.read_sql_query(select, conn)
display(dt)

In [ ]:
sofifa_id = []
sn = []
ln = []
age = []
dob = []
height = []
weight = []
country = []
club = []
overall = []
potential = []
value_eur = []
wage_eur = []
player_positions = []
preferred_foot = []
international_reputation =[]
weak_foot=[]
skill_moves=[]
work_rate=[]
release_clause_eur=[]
player_tags=[]
with open('players_18.txt') as file :
    for line in file:
        if not line.strip():
            continue 
        line = line.strip().split('\t')
        sofifa_id.append(line[0])
        sn.append(line[2])
        ln.append(line[3])
        age.append(line[4])
        dob.append(line[5])
        height.append(line[6])
        weight.append(line[7])
        country.append(line[8])
        club.append(line[9])
        overall.append(line[10])
        potential.append(line[11])
        value_eur.append(line[12])
        wage_eur.append(line[13])
        player_positions.append(line[14])
        preferred_foot.append(line[15])
        international_reputation.append(line[16])
        weak_foot.append(line[17])
        skill_moves.append(line[18])
        work_rate.append(line[19])
        release_clause_eur.append(line[22])
        player_tags.append(line[23])

In [ ]:
table1=list(zip(sofifa_id,sn,ln))
create_table_sql1 = """CREATE TABLE [Player_Details] (
            [sofifa_id] Real not null primary key,
            [short_name] Text not null,
            [long_name] Text not null
        );"""
database_filename = 'project.db'  
conn = sqlite3.connect(database_filename)
create_table(conn, create_table_sql1,'Player_Details')


def insert_Player(conn, values):
    sql = '''INSERT INTO Player_Details(sofifa_id,short_name,long_name)VALUES(?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
    for r in table1:
        insert_Player(conn,(r[0],r[1],r[2]))
sel1 = 'select * from Player_Details'
table1 = pd.read_sql_query(sel1, conn)
display(table1)

In [ ]:
table2=list(country)
create_table_sql2 = """CREATE TABLE [Player_Country] (
            [Country_id] Integer not null primary key AUTOINCREMENT,
            [nationality] Text not null
        );"""
database_filename = 'project.db' 
conn = sqlite3.connect(database_filename)
create_table(conn, create_table_sql2,'Player_Country')
def insert_country(conn, values):
    sql = '''INSERT INTO Player_Country(nationality)VALUES(?)'''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
    for new in table2:
        insert_country(conn,(new,))
sel2 = 'select * from Player_Details'
table2 = pd.read_sql_query(sel2, conn)
display(table2)

In [ ]:
table3=list(club)
create_table_sql3 = """CREATE TABLE [Player_Club] (
            [Club_id] Integer not null primary key AUTOINCREMENT,
            [club] Text not null
        );"""
database_filename = 'project.db'
conn = sqlite3.connect(database_filename)
create_table(conn, create_table_sql3,'Player_Club')


def insert_club(conn, values):
    sql = '''INSERT INTO Player_Club(club)VALUES(?)'''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
    for new in table3:
        insert_club(conn,(new,))
sel3 = 'select * from Player_Details'
table3 = pd.read_sql_query(sel3, conn)
display(table3)

In [ ]:
table4=list(zip(sofifa_id,age,height,weight))
create_table_sql4 = """CREATE TABLE [Player_personality] (
            [sofifa_id] Real not null primary key,
            [age] integer not null,
            [height_cm] integer not null,
            [weight_kg] integer not null
        );"""
database_filename = 'project.db'  
conn = sqlite3.connect(database_filename)
create_table(conn, create_table_sql4,'Player_personality')


def insert_personality(conn, values):
    sql = '''INSERT INTO Player_personality(sofifa_id,age,height_cm,weight_kg)VALUES(?,?,?,?)'''
    #step2_create_region_to_regionid_dictionary(normalized_database_filename)
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
    for r in table4:
        insert_personality(conn,((r[0],r[1],r[2],r[3])))
sel4 = 'select * from Player_Details'
table4 = pd.read_sql_query(sel4, conn)
display(table4)

In [ ]:
table5=list(zip(sofifa_id,value_eur,wage_eur,player_positions,international_reputation))
create_table_sql5 = """CREATE TABLE [Player_value] (
            [sofifa_id] Real not null primary key,
            [value_eur] integer not null,
            [wage_eur] integer not null,
            [player_positions] Text not null,
            [international_reputation] integer not null,
            [Club_id] integer,
            FOREIGN KEY(Club_id) REFERENCES Player_Club(Club_id)
        );"""
database_filename = 'project.db'
conn = sqlite3.connect(database_filename)
create_table(conn, create_table_sql5,'Player_value')


def insert_value(conn, values):
    sql = '''INSERT INTO Player_value(sofifa_id,value_eur,wage_eur,player_positions,international_reputation)VALUES(?,?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
    for r in table5:
        insert_value(conn,((r[0],r[1],r[2],r[3],r[4])))
sel5 = 'select * from Player_Details'
table5 = pd.read_sql_query(sel5, conn)
display(table5)

## 2) Understanding the data 

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
dt.info()

In [ ]:
dt.shape

In [ ]:
dt.head()

In [ ]:
table1.info()

In [ ]:
table1.shape

In [ ]:
table1.head()

In [ ]:
table2.info()

In [ ]:
table2.shape

In [ ]:
table2.head()

In [ ]:
table3.info()

In [ ]:
table3.shape

In [ ]:
table3.head()

In [ ]:
table4.info()

In [ ]:
table4.shape

In [ ]:
table4.head()

In [ ]:
table5.info()

In [ ]:
table5.shape

In [ ]:
table5.head()

##  3) Visualising the data

### AGE

In [ ]:
plt.figure(figsize=(16, 30))
sns.displot(data=df, x="age", kind="kde", height = 7)

 We understand from the above graph that max players of the age grop of 20 - 28

### AGE VS VALUE

In [ ]:
sns.set(rc = {'figure.figsize':(10,8)})  
g = sns.relplot(data=df, x="age", y="value_eur",height = 7)
g.ax.axline(xy1=(10, 2), slope=.2, color="b", dashes=(5, 2))

from the above graph  We can that there is a steep decline of value after or around 30 . 

### Overall

In [ ]:
sns.displot(data=df, x="overall",   kind="kde",height = 7)

### Overall VS Value of a Player

In [ ]:
g = sns.relplot(data=df, x="overall", y="value_eur"  ,height = 7  )
g.ax.axline(xy1=(10, 10), slope=.2, color="b", dashes=(5, 2))

 AS potential increases value increases

### Potential 

In [ ]:
sns.displot(data=df, x="potential",   kind="kde",height = 7)

### Potential vs Value

In [ ]:
g = sns.relplot(data=df, x="potential", y="value_eur" ,height = 7  )
g.ax.axline(xy1=(10, 10), slope=.2, color="b", dashes=(5, 2))

 High potential = High Value

### WAGE

In [ ]:
import matplotlib.pyplot as plt
scatter = sns.displot(data=df, x="wage_eur",   kind="kde",height = 7 ,)
plt.xlim(0, 200000)
plt.show()

In [ ]:
ax = sns.relplot(data=df, x="wage_eur", y="value_eur" ,height = 6  ,)
g.ax.axline(xy1=(9, 7), slope=.2, color="b", dashes=(5, 2))

Understanding from the graph  Directly praportional to wage.

 ### International_reputation

In [ ]:
sns.displot(data=df, x="international_reputation",   kind="kde",height = 7 ,)

### Overall VS Value of a Player VS International_reputation

In [ ]:
ax = sns.relplot(data=df, x="overall", y="value_eur"  ,hue="international_reputation",height = 7  ,ax=ax)
g.ax.axline(xy1=(10, 10), slope=.2, color="b", dashes=(5, 2))

 Higher International reputation with high overall means higher value of player 

 ### Player_positions

In [ ]:
type(dt)

In [ ]:
dt['player_positions'].unique()

Filtering only the main positions of players .

In [ ]:
for i in range(len(dt.index)): #Taking best position only, not considering multiple positions
     dt['player_positions'][i] = dt.player_positions[i].split(',',1)[0]

In [ ]:
dt['player_positions'].unique()

Right and left position won't contribute to the value so converting it into one.

In [ ]:
for i in range (len(dt.index)):
          
          if dt['player_positions'][i] == 'LB' or dt['player_positions'][i] == 'RB' or dt['player_positions'][i] == 'LWB' or dt['player_positions'][i] == 'RWB':
            dt['player_positions'][i] = 'LB'
          
          if dt['player_positions'][i] == 'RM' or dt['player_positions'][i] == 'LM':
            dt['player_positions'][i] = 'LM'
          
        
          if dt['player_positions'][i] == 'LW' or dt['player_positions'][i] == 'RW':
            dt['player_positions'][i] = 'LW'
        

In [ ]:
dt['player_positions'].unique()

Eventually coverting into 'Attacker', 'GK', 'Defender ', 'MID' . 

In [ ]:
for i in range (len(dt.index)):
    if dt['player_positions'][i] == 'LW' or dt['player_positions'][i] == 'ST' or dt['player_positions'][i] == 'CAM' or dt['player_positions'][i] == 'CF':
        dt['player_positions'][i] = 'Attacker'
    if dt['player_positions'][i] == 'CM' or dt['player_positions'][i] == 'LM' or dt['player_positions'][i] == 'CDM':
        dt['player_positions'][i] = 'MID'   
    if dt['player_positions'][i] == 'CB' or dt['player_positions'][i] == 'LB' :
        dt['player_positions'][i] = 'Defender '   

In [ ]:
dt['player_positions'].unique()

In [ ]:
plt.bar(dt.player_positions, dt.value_eur, color = (0.5,0.1,0.5,0.6))
plt.show()

As we can see Attackers are values higher comapred to other positions .

In [ ]:
dt['contract_valid_until'] = dt['contract_valid_until'].replace('' ,np.nan)
dt['release_clause_eur'] = dt['release_clause_eur'].replace('' ,np.nan)
dt['player_positions'] = dt['player_positions'].replace('' ,np.nan)

### Contract Valid

In [ ]:
dt.info()

In [ ]:
dt.contract_valid_until.unique()

In [ ]:
# dt['contract_valid_until'] = dt['contract_valid_until'].replace('' ,'nan')

In [ ]:
# for i in range (len(dt.index)):
#     if dt['contract_valid_until'][i] == [' ']:
#         dt['contract_valid_until'][i].replace(0)
# dt.contract_valid_until.unique()

In [ ]:
ax = sns.relplot(data=dt, x="contract_valid_until", y="value_eur" ,height = 7  ,)
g.ax.axline(xy1=(9, 7), slope=.2, color="b", dashes=(5, 2))

Contracts with less than 1 year have low value .

### Skill moves 

In [ ]:
ax = sns.relplot(data=dt, x="skill_moves", y="value_eur" ,height = 7  ,)
g.ax.axline(xy1=(9, 7), slope=.2, color="b", dashes=(5, 2))

High value players have High skill moves value.

## 4) PreProcessing Data 

Replacing na values 

In [ ]:
dats = dt.replace(np.nan, 0)

In [ ]:
dats.info()

In [ ]:
dats.dtypes

Converting Player position using one hot encoding method 

In [ ]:
dats = pd.get_dummies(dats, columns = ['player_positions'])

In [ ]:
dats.shape

In [ ]:
dats.head()

## 5) Scaling and dividing data into test and training 

In [ ]:
y = dats['value_eur']
dats.drop('value_eur' , axis=1 , inplace = True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dats, y, test_size=0.2 , random_state=43)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

In [ ]:
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax

## 6) Applying Regression models 

### Linear Regression Model 

In [ ]:
regressor=LinearRegression()
regressor.fit(X_train_minmax,y_train)

In [ ]:
y_pred=regressor.predict(X_test_minmax)
y_pred
y_test

In [ ]:
SQR_linearreg = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
R2_linearreg = r2_score(y_test, y_pred)
cv_r2_scores_lr = cross_val_score(regressor, dats, y, cv=5,scoring='r2')
print(cv_r2_scores_lr)
print("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores_lr)))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred), max(y_test))
p2 = min(min(y_pred), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.title('linear_regression prediction model')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

### Kflod 

In [ ]:
model = KNeighborsClassifier()
model.fit(X_train_minmax, y_train)

In [ ]:
actual_y, predict_y = list(), list()
y_predict = model.predict(X_test_minmax)
actual_y.extend(y_test)
predict_y.extend(y_predict)

In [ ]:
SQR_KF = np.sqrt(metrics.mean_squared_error(actual_y, predict_y))
R2_KF = r2_score(actual_y, predict_y)
cv_r2_scores_kf = cross_val_score(model, dats, y, cv=5,scoring='r2')
print(cv_r2_scores_kf)
print("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores_kf)))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(actual_y, predict_y, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predict_y), max(actual_y))
p2 = min(min(predict_y), min(actual_y))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.title('kfold prediction model')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

### SVC - C-Support Vector Classification.

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_minmax, y_train)
Y_pred2 = classifier.predict(X_test_minmax)

In [ ]:
SQT_SVC = np.sqrt(metrics.mean_squared_error(y_test, Y_pred2))
R2_SVC = r2_score(y_test, Y_pred2)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, Y_pred2, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(Y_pred2), max(y_test))
p2 = min(min(Y_pred2), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.title('SVC prediction model')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

 ### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier2.fit(X_train_minmax, y_train)
Y_pred3 = classifier2.predict(X_test_minmax)

In [ ]:
SQR_RFC = np.sqrt(metrics.mean_squared_error(y_test, Y_pred3))
R2_RFC = r2_score(y_test, Y_pred3)
cv_r2_scores_rfc = cross_val_score(classifier2, dats, y, cv=5,scoring='r2')
print(cv_r2_scores_rfc)
print("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores_rfc)))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, Y_pred3, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(Y_pred3), max(y_test))
p2 = min(min(Y_pred3), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.title('Random_Forest prediction model')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

### Decision Tree 

In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
clf = clf.fit(X_train_minmax, y_train)
y_pred4 = clf.predict(X_test_minmax)

In [ ]:
SQR_DT = np.sqrt(metrics.mean_squared_error(y_test, y_pred4))
R2_DT = r2_score(y_test, y_pred4)
cv_r2_scores_DT = cross_val_score(clf, dats, y, cv=5,scoring='r2')
print(cv_r2_scores_DT)
print("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores_DT)))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred4, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred4), max(y_test))
p2 = min(min(y_pred4), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.title('Decision_Tree prediction model')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

 ## statistics of each model

In [ ]:
header = ['Linear Regression' , 'Kfold' , 'SVC' ,'Random Forest' ,'Desicion Tree']
data = [np.log(SQR_linearreg) , np.log(SQR_KF), np.log(SQT_SVC) , np.log(SQR_RFC) ,np.log( SQR_DT)]
stat_analysis = pd.DataFrame(data = data , index = header )
stat_analysis.rename(columns = {0:'Square_root'}, inplace = True)
stat_analysis['R2_Score'] = [R2_linearreg , R2_KF ,R2_SVC , R2_RFC , R2_DT ]
stat_analysis

# Conclusion

We made five models and compared their r2 scores to understand their efficency . tho all models presented us with good results  Linear regression model and Desicion Tree model handled various features much better comparitivley .

# Reference 

https://mkzia.github.io/eas503-notes/intro.html


https://www.kaggle.com/datasets/stefanoleone992/fifa-20-complete-player-dataset


https://scikit-learn.org/stable/supervised_learning.html

# Acknowledgement 

We would like to sincerely thank our Prof. Mohamad Zia for the guidance thought this course helping us accomplish this project.